![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Detect a Car Going in the Wrong Direction

This notebook is an example how to use DeGirum PySDK to detect a car going in the wrong direction
using object detection, object tracking, line cross counting, and event detection.
When an event is detected, the notification is sent to the notification service of your choice
and a video clip around that event is uploaded to S3-compatible storage of your choice.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [1]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify video file name, model name, and other options here

In [2]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# cross_line: line that marks the lane crossing. Format: [x_start, y_start, x_end, y_end].
# It should be oriented so cars moving in wrong direction would cross it from left to right
# when looking towards line end.
# clip_duration: duration of the video clip to save, in frames
# storage_config: configuration for storing the results in S3-compatible storage
# notification_config: Apprise-compatible configuration for sending notifications
# (see https://github.com/caronc/apprise for details)
import degirum as dg, degirum_tools

hw_location = "@cloud"
model_zoo_url = "degirum/public"
model_name = "yolo_v5n_car_det--512x512_quant_n2x_orca1_1"
video_source = (
    "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Traffic.mp4"
)
cross_line = [(800, 180, 900, 80)]  # [x_start, y_start, x_end, y_end]

clip_duration = 30  # frames

storage_config = degirum_tools.ObjectStorageConfig(
    endpoint="./temp",  # endpoint url, or path to local folder for local storage
    access_key="",  # access key for S3-compatible storage
    secret_key="",  # secret key for S3-compatible storage
    bucket="car_wrong_direction",  # bucket name for S3-compatible storage or subdirectory name for local storage
)


notification_config = degirum_tools.notification_config_console

#### The rest of the cells below should run without any modifications

In [ ]:
# load model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=hw_location,
    zoo_url=model_zoo_url,
    token=degirum_tools.get_token(),
    overlay_color=[(255, 0, 0)],
    overlay_line_width=1,
    overlay_show_labels=False,
)

# bbox anchor point for object tracker and line counter
anchor = degirum_tools.AnchorPoint.CENTER

# create object tracker
object_tracker = degirum_tools.ObjectTracker(
    track_thresh=0.35,
    track_buffer=100,
    match_thresh=0.9999,
    trail_depth=20,
    anchor_point=anchor,
)

# create line crossing counter
line_counter = degirum_tools.LineCounter(
    cross_line,
    anchor,
    accumulate=False,
    show_overlay=True,
    annotation_color=(255, 255, 0),
)

event_name = "car_moving_wrong_direction"

# create event detector: car crosses a line
linecross_detector = degirum_tools.EventDetector(
    f"""
    Trigger: {event_name}
    when: LineCount
    with:
        directions: [right]
    is greater than: 0
    during: [1, frame]
    for at least: [1, frame]
    """,
    show_overlay=False,
)

# create event notifier: car crosses line in wrong direction
annotation_pos = cross_line[0][:2]
linecross_notifier = degirum_tools.EventNotifier(
    "Wrong Direction",
    event_name,
    message="${time}: ${result.events_detected} (${url})",
    annotation_pos=annotation_pos,
    annotation_color=(255, 0, 0),
    annotation_cool_down=1.0,
    notification_config=notification_config,
    clip_save=True,
    clip_duration=clip_duration,
    clip_pre_trigger_delay=clip_duration // 2,
    storage_config=storage_config,
)

# attach zone counter to model
degirum_tools.attach_analyzers(
    model,
    [
        object_tracker,
        line_counter,
        linecross_detector,
        linecross_notifier,
    ],
)

# run inference and display results
with degirum_tools.Display() as display:
    for inference_result in degirum_tools.predict_stream(model, video_source):
        display.show(inference_result)
        # print(inference_result.line_counts[0].right, inference_result.events_detected, inference_result.notifications)

# detach analyzers from model to finalize them
degirum_tools.attach_analyzers(model, None)